In [1]:
import pandas as pd
import numpy as np
import math 
from sklearn.linear_model import LogisticRegression

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
# Unpack all the dataset
bureau = pd.read_csv(r"D:\Project Data Prep\Version 1\cleaned_bureau_ver1.csv")
credit_card_balance = pd.read_csv(r"D:\Project Data Prep\Version 1\dseb63_credit_card_balance_ver1 1.csv")
installment_payments = pd.read_csv(r"D:\Project Data Prep\Version 1\installment_payments_ver1 1.csv")
pos_cash = pd.read_csv(r"D:\Project Data Prep\Version 1\POS_CASH_CLEANED_xulioutlier_truoc_roi_fill_null ver 1.1.csv")
previous_application = pd.read_csv(r"D:\Project Data Prep\Version 1\previous_application ver1.csv")
bureau_balance = pd.read_csv(r"C:\Users\ADMIN\Downloads\dseb-64-data-preparation-final-project\dseb63_final_project_DP_dataset\dseb63_final_project_DP_dataset\dseb63_bureau_balance.csv")

In [4]:
# Group by using SK_ID_CURR, then create columns based on informations about each applicants
num_aggregations = {
    'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
    'DAYS_CREDIT_UPDATE': ['mean'],
    'CREDIT_DAY_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_MAX_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
    'AMT_ANNUITY': ['max', 'mean'],
    'CNT_CREDIT_PROLONG': ['sum']}
bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations})
bureau_agg.columns = pd.Index(['BUR_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()]) #Change the name of columns
bureau_agg.head()

,BUR_DAYS_CREDIT_MIN,BUR_DAYS_CREDIT_MAX,BUR_DAYS_CREDIT_MEAN,BUR_DAYS_CREDIT_VAR,BUR_DAYS_CREDIT_ENDDATE_MIN,BUR_DAYS_CREDIT_ENDDATE_MAX,BUR_DAYS_CREDIT_ENDDATE_MEAN,BUR_DAYS_CREDIT_UPDATE_MEAN,BUR_CREDIT_DAY_OVERDUE_MAX,BUR_CREDIT_DAY_OVERDUE_MEAN,...,BUR_AMT_CREDIT_SUM_SUM,BUR_AMT_CREDIT_SUM_DEBT_MAX,BUR_AMT_CREDIT_SUM_DEBT_MEAN,BUR_AMT_CREDIT_SUM_DEBT_SUM,BUR_AMT_CREDIT_SUM_OVERDUE_MEAN,BUR_AMT_CREDIT_SUM_LIMIT_MEAN,BUR_AMT_CREDIT_SUM_LIMIT_SUM,BUR_AMT_ANNUITY_MAX,BUR_AMT_ANNUITY_MEAN,BUR_CNT_CREDIT_PROLONG_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,-63,-63,-63.000000,NaN,237.0,237.0,237.000000,-28.000000,0,0.0,...,85513.50,77566.5,77566.5,77566.5,0.0,0.0,0.0,8553.452346,8553.452346,0
1,-2348,-2348,-2348.000000,NaN,-2044.0,-2044.0,-2044.000000,-18.000000,0,0.0,...,28350.00,0.0,0.0,0.0,0.0,0.0,0.0,8553.452346,8553.452346,0
2,-2901,-30,-810.333333,1.222703e+06,-2597.0,274.0,-572.166667,-566.916667,0,0.0,...,254341.08,58333.5,16069.5,96417.0,0.0,0.0,0.0,8553.452346,8553.452346,0
3,-2865,-116,-1131.428571,1.260530e+06,-2683.0,502.0,-658.714286,-730.857143,0,0.0,...,483952.50,99135.0,20029.5,140206.5,0.0,0.0,0.0,8553.452346,8553.452346,0
4,-1056,-313,-773.333333,1.082075e+05,-509.0,1148.0,474.500000,-63.333333,0,0.0,...,2512755.00,99135.0,49567.5,297405.0,0.0,0.0,0.0,8553.452346,8553.452346,0


In [5]:
# Adding aggregates for closed credit
closed = bureau[bureau['CREDIT_ACTIVE'] == 'Closed']
closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
bur_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
bur_agg.head()

,BUR_DAYS_CREDIT_MIN,BUR_DAYS_CREDIT_MAX,BUR_DAYS_CREDIT_MEAN,BUR_DAYS_CREDIT_VAR,BUR_DAYS_CREDIT_ENDDATE_MIN,BUR_DAYS_CREDIT_ENDDATE_MAX,BUR_DAYS_CREDIT_ENDDATE_MEAN,BUR_DAYS_CREDIT_UPDATE_MEAN,BUR_CREDIT_DAY_OVERDUE_MAX,BUR_CREDIT_DAY_OVERDUE_MEAN,...,CLOSED_AMT_CREDIT_SUM_SUM,CLOSED_AMT_CREDIT_SUM_DEBT_MAX,CLOSED_AMT_CREDIT_SUM_DEBT_MEAN,CLOSED_AMT_CREDIT_SUM_DEBT_SUM,CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_SUM,CLOSED_AMT_ANNUITY_MAX,CLOSED_AMT_ANNUITY_MEAN,CLOSED_CNT_CREDIT_PROLONG_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,-63,-63,-63.000000,NaN,237.0,237.0,237.000000,-28.000000,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-2348,-2348,-2348.000000,NaN,-2044.0,-2044.0,-2044.000000,-18.000000,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2901,-30,-810.333333,1.222703e+06,-2597.0,274.0,-572.166667,-566.916667,0,0.0,...,153068.58,0.0,0.0,0.0,0.0,0.0,0.0,8553.452346,8553.452346,0.0
3,-2865,-116,-1131.428571,1.260530e+06,-2683.0,502.0,-658.714286,-730.857143,0,0.0,...,271687.50,0.0,0.0,0.0,0.0,0.0,0.0,8553.452346,8553.452346,0.0
4,-1056,-313,-773.333333,1.082075e+05,-509.0,1148.0,474.500000,-63.333333,0,0.0,...,1098877.50,0.0,0.0,0.0,0.0,0.0,0.0,8553.452346,8553.452346,0.0


bureau_balance

In [6]:
# Aggregation for bureau balance
bb_aggregations = {'MONTHS_BALANCE': ['min', 'mean', 'max', 'size']}
bb_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(bb_aggregations)
bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()]) # Change columns name
bb_agg.head()

,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MEAN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_SIZE
SK_ID_BUREAU,,,,
5001709,-96,-48.0,0,97
5001710,-82,-41.0,0,83
5001711,-3,-1.5,0,4
5001712,-18,-9.0,0,19
5001713,-21,-10.5,0,22


CREDIT_CARD_BALANCE

In [7]:
# Aggregations for credit card balance
cred_aggregations = {'MONTHS_BALANCE': ['min', 'mean', 'max'],
                    'AMT_BALANCE': ['min', 'mean', 'max']}
cred_agg = credit_card_balance.groupby('SK_ID_CURR').agg(cred_aggregations)
cred_agg.columns = pd.Index(['CRED_' + e[0] + "_" + e[1].upper() for e in cred_agg.columns.tolist()])
# Count of total entries
cred_agg['CRED_COUNT'] = credit_card_balance.groupby('SK_ID_CURR').size()
cred_agg.head()

,CRED_MONTHS_BALANCE_MIN,CRED_MONTHS_BALANCE_MEAN,CRED_MONTHS_BALANCE_MAX,CRED_AMT_BALANCE_MIN,CRED_AMT_BALANCE_MEAN,CRED_AMT_BALANCE_MAX,CRED_COUNT
SK_ID_CURR,,,,,,,
0,-8,-4.5,-1,0.0,6735.504375,36569.430,8
1,-9,-5.0,-1,15271.2,147012.840000,226013.850,9
3,-11,-6.0,-1,0.0,124425.875455,140704.965,11
7,-18,-9.5,-1,0.0,0.000000,0.000,18
9,-11,-6.0,-1,0.0,428.318182,4711.500,11


INSTALLMENT_PAYMENT

In [8]:
# Finding percentage and difference paid in each installment
installment_payments['PAYMENT_PERC'] = installment_payments['AMT_PAYMENT'] / installment_payments['AMT_INSTALLMENT']
installment_payments['PAYMENT_DIFF'] = installment_payments['AMT_INSTALLMENT'] - installment_payments['AMT_PAYMENT']
# Finding days past due (DPD) and days before due (DBD)
installment_payments['DPD'] = installment_payments['DAYS_ENTRY_PAYMENT'] - installment_payments['DAYS_INSTALLMENT']
installment_payments['DBD'] = installment_payments['DAYS_INSTALLMENT'] - installment_payments['DAYS_ENTRY_PAYMENT']
installment_payments['DPD'] = installment_payments['DPD'].apply(lambda x: x if x > 0 else 0)
installment_payments['DBD'] = installment_payments['DBD'].apply(lambda x: x if x > 0 else 0)
# Creating aggregations
inst_aggregations = {
    'NUM_INSTALLMENT_VERSION': ['nunique'],
    'DPD': ['max', 'mean', 'sum'],
    'DBD': ['max', 'mean', 'sum'],
    'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
    'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
    'AMT_INSTALLMENT': ['max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']}
inst_agg = installment_payments.groupby('SK_ID_CURR').agg(inst_aggregations)
inst_agg.columns = pd.Index(['INST_' + e[0] + "_" + e[1].upper() for e in inst_agg.columns.tolist()])
# Count of installments accounts
inst_agg['INST_COUNT'] = installment_payments.groupby('SK_ID_CURR').size()
inst_agg.head()

,INST_NUM_INSTALLMENT_VERSION_NUNIQUE,INST_DPD_MAX,INST_DPD_MEAN,INST_DPD_SUM,INST_DBD_MAX,INST_DBD_MEAN,INST_DBD_SUM,INST_PAYMENT_PERC_MAX,INST_PAYMENT_PERC_MEAN,INST_PAYMENT_PERC_SUM,...,INST_AMT_INSTALLMENT_MEAN,INST_AMT_INSTALLMENT_SUM,INST_AMT_PAYMENT_MIN,INST_AMT_PAYMENT_MAX,INST_AMT_PAYMENT_MEAN,INST_AMT_PAYMENT_SUM,INST_DAYS_ENTRY_PAYMENT_MAX,INST_DAYS_ENTRY_PAYMENT_MEAN,INST_DAYS_ENTRY_PAYMENT_SUM,INST_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,3,0,0.000,0,64,10.900000,327,1.00000,0.998154,29.944622,...,7585.66350,227569.905,141.570,34312.500,7521.13350,225634.005,-5,-150.433333,-4513,30
1,3,29,2.075,83,30,4.000000,160,1.00000,0.849539,33.981541,...,15427.90350,617116.140,120.780,34312.500,11512.31625,460492.650,-18,-622.050000,-24882,40
3,3,0,0.000,0,99,11.581395,498,2.51059,1.050096,45.154123,...,5503.11907,236634.120,78.435,34312.500,5692.18186,244763.820,-5,-228.232558,-9814,43
4,1,0,0.000,0,23,17.166667,103,1.00000,1.000000,6.000000,...,8953.52250,53721.135,8933.985,8957.430,8953.52250,53721.135,-2230,-2299.166667,-13795,6
5,1,0,0.000,0,17,9.200000,92,1.00000,1.000000,10.000000,...,9818.64450,98186.445,9799.650,9820.755,9818.64450,98186.445,-1808,-1944.200000,-19442,10


PREVIOUS APPLICATION

In [9]:
# Percentage of value ask / value received percentage
previous_application['APP_CREDIT_PERC'] = previous_application['AMT_APPLICATION'] / previous_application['AMT_CREDIT']
# Aggregations for previous applications
prev_aggregations = {
    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_APPLICATION': ['min', 'max', 'mean'],
    'AMT_CREDIT': ['min', 'max', 'mean'],
    'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
    'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
    'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
    'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['mean', 'sum']}
prev_agg = previous_application.groupby('SK_ID_CURR').agg(prev_aggregations)
prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
# Previous Applications: Approved Applications only
approved = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Approved']
approved_agg = approved.groupby('SK_ID_CURR').agg(prev_aggregations)
approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
# Previous Applications: Refused Applications - only numerical features
refused = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Refused']
refused_agg = refused.groupby('SK_ID_CURR').agg(prev_aggregations)
refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
prev_agg.head()

,PREV_AMT_ANNUITY_MIN,PREV_AMT_ANNUITY_MAX,PREV_AMT_ANNUITY_MEAN,PREV_AMT_APPLICATION_MIN,PREV_AMT_APPLICATION_MAX,PREV_AMT_APPLICATION_MEAN,PREV_AMT_CREDIT_MIN,PREV_AMT_CREDIT_MAX,PREV_AMT_CREDIT_MEAN,PREV_APP_CREDIT_PERC_MIN,...,REFUSED_HOUR_APPR_PROCESS_START_MAX,REFUSED_HOUR_APPR_PROCESS_START_MEAN,REFUSED_RATE_DOWN_PAYMENT_MIN,REFUSED_RATE_DOWN_PAYMENT_MAX,REFUSED_RATE_DOWN_PAYMENT_MEAN,REFUSED_DAYS_DECISION_MIN,REFUSED_DAYS_DECISION_MAX,REFUSED_DAYS_DECISION_MEAN,REFUSED_CNT_PAYMENT_MEAN,REFUSED_CNT_PAYMENT_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
0,2250.000,11180.38500,6659.137500,25200.0,120658.5,78399.000000,21928.5,120658.5,77853.750000,1.000000,...,15.0,15.0,0.0,0.0,0.0,-121.0,-121.0,-121.0,24.0,24.0
1,2250.000,25996.36500,13688.302617,0.0,337500.0,68303.571429,0.0,384277.5,74634.428571,0.878272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2457.675,24577.42500,9705.645000,16420.5,225000.0,85230.123750,14170.5,239850.0,87738.750000,0.938086,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3404.520,14272.70083,8013.459443,0.0,135000.0,41482.500000,0.0,135000.0,43401.000000,0.904489,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8957.430,8957.43000,8957.430000,47250.0,47250.0,47250.000000,44631.0,44631.0,44631.000000,1.058681,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Aggregations for POS
pos_aggregations = {
    'MONTHS_BALANCE': ['max', 'mean', 'size'],
    'SK_DPD': ['max', 'mean'],
    'SK_DPD_DEF': ['max', 'mean']}
pos_agg = pos_cash.groupby('SK_ID_CURR').agg(pos_aggregations)
pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
# Count of pos cash accounts
pos_agg['POS_COUNT'] = pos_cash.groupby('SK_ID_CURR').size()
pos_agg.head()

,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,
0,-1.0,-7.466667,15,0,0.0,0,0.0,15
1,-12.0,-32.086957,23,0,0.0,0,0.0,23
2,-23.0,-55.147059,34,0,0.0,0,0.0,34
3,-5.0,-12.428571,14,0,0.0,0,0.0,14
4,-73.0,-76.000000,7,0,0.0,0,0.0,7


In [11]:
train = pd.read_csv(r"C:\Users\ADMIN\Downloads\dseb-64-data-preparation-final-project\dseb63_final_project_DP_dataset\dseb63_final_project_DP_dataset\dseb63_application_train.csv")

In [12]:
train = train.join(bur_agg, how = "left", on = 'SK_ID_CURR')
train = train.join(bb_agg, how = "left", on = 'SK_ID_CURR')
train = train.join(cred_agg, how = "left", on = 'SK_ID_CURR')
train = train.join(inst_agg, how = "left", on = 'SK_ID_CURR')
train = train.join(prev_agg, how = "left", on = 'SK_ID_CURR')
train = train.join(pos_agg, how = "left", on = 'SK_ID_CURR')
train.head()

,Unnamed: 0,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REFUSED_CNT_PAYMENT_MEAN,REFUSED_CNT_PAYMENT_SUM,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_COUNT
0,0,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,NaN,NaN,-18.0,-43.785714,28.0,0.0,0.0,0.0,0.0,28.0
1,1,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,42.0,42.0,-1.0,-9.619048,21.0,0.0,0.0,0.0,0.0,21.0
2,2,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,NaN,NaN,-1.0,-33.636364,66.0,0.0,0.0,0.0,0.0,66.0
3,3,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,...,NaN,NaN,-2.0,-43.662651,83.0,0.0,0.0,0.0,0.0,83.0
4,4,0,Cash loans,M,Y,Y,0,360000.0,1530000.0,42075.0,...,NaN,NaN,-25.0,-30.000000,11.0,0.0,0.0,0.0,0.0,11.0


# Clean the training and test dataset

In [14]:
# Read test file
test = pd.read_csv(r"C:\Users\ADMIN\Downloads\dseb-64-data-preparation-final-project\dseb63_final_project_DP_dataset\dseb63_final_project_DP_dataset\dseb63_application_test.csv")

In [ ]:
# drop train columns that contains all null values
train.dropna(axis = 1, how = 'all', inplace = True)

In [15]:
test = test.join(bur_agg, how = "left", on = 'SK_ID_CURR')
test = test.join(bb_agg, how = "left", on = 'SK_ID_CURR')
test = test.join(cred_agg, how = "left", on = 'SK_ID_CURR')
test = test.join(inst_agg, how = "left", on = 'SK_ID_CURR')
test = test.join(prev_agg, how = "left", on = 'SK_ID_CURR')
test = test.join(pos_agg, how = "left", on = 'SK_ID_CURR')

In [ ]:
# Drop colums that contain all null values
test.dropna(axis = 1, how = 'all', inplace = True)

In [17]:
test.head()

,Unnamed: 0,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,REFUSED_CNT_PAYMENT_MEAN,REFUSED_CNT_PAYMENT_SUM,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_COUNT
0,0,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,418500.0,...,2.0,6.0,-54.0,-64.565217,23.0,0.0,0.0,0.0,0.0,23.0
1,1,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,1179000.0,...,6.0,12.0,-24.0,-46.977273,44.0,0.0,0.0,0.0,0.0,44.0
2,2,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,495000.0,...,15.0,15.0,-5.0,-25.476190,21.0,0.0,0.0,0.0,0.0,21.0
3,3,Cash loans,F,N,Y,0,247500.0,254700.0,24939.0,225000.0,...,NaN,NaN,-42.0,-54.000000,25.0,0.0,0.0,0.0,0.0,25.0
4,4,Cash loans,M,N,Y,0,112500.0,308133.0,15862.5,234000.0,...,6.0,18.0,-4.0,-39.461538,26.0,0.0,0.0,0.0,0.0,26.0


In [18]:
# Drop unwanted column
test.drop('Unnamed: 0', axis = 1, inplace = True)

In [19]:
# Choose training set x and target y

y = train['TARGET']
x = train.iloc[:, 2:]
x.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,REFUSED_CNT_PAYMENT_MEAN,REFUSED_CNT_PAYMENT_SUM,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_COUNT
0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,...,NaN,NaN,-18.0,-43.785714,28.0,0.0,0.0,0.0,0.0,28.0
1,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,...,42.0,42.0,-1.0,-9.619048,21.0,0.0,0.0,0.0,0.0,21.0
2,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,...,NaN,NaN,-1.0,-33.636364,66.0,0.0,0.0,0.0,0.0,66.0
3,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,454500.0,"Spouse, partner",...,NaN,NaN,-2.0,-43.662651,83.0,0.0,0.0,0.0,0.0,83.0
4,Cash loans,M,Y,Y,0,360000.0,1530000.0,42075.0,1530000.0,Unaccompanied,...,NaN,NaN,-25.0,-30.000000,11.0,0.0,0.0,0.0,0.0,11.0


In [20]:
test.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,REFUSED_CNT_PAYMENT_MEAN,REFUSED_CNT_PAYMENT_SUM,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_COUNT
0,Cash loans,M,Y,N,2,207000.0,465457.5,52641.0,418500.0,Unaccompanied,...,2.0,6.0,-54.0,-64.565217,23.0,0.0,0.0,0.0,0.0,23.0
1,Cash loans,F,Y,Y,0,247500.0,1281712.5,48946.5,1179000.0,Unaccompanied,...,6.0,12.0,-24.0,-46.977273,44.0,0.0,0.0,0.0,0.0,44.0
2,Cash loans,F,Y,N,0,202500.0,495000.0,39109.5,495000.0,Unaccompanied,...,15.0,15.0,-5.0,-25.476190,21.0,0.0,0.0,0.0,0.0,21.0
3,Cash loans,F,N,Y,0,247500.0,254700.0,24939.0,225000.0,Unaccompanied,...,NaN,NaN,-42.0,-54.000000,25.0,0.0,0.0,0.0,0.0,25.0
4,Cash loans,M,N,Y,0,112500.0,308133.0,15862.5,234000.0,Unaccompanied,...,6.0,18.0,-4.0,-39.461538,26.0,0.0,0.0,0.0,0.0,26.0


In [21]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [22]:
# Replace infinite values with null values
test.replace([np.inf, -np.inf], np.nan, inplace=True)

In [23]:
# Fill missing values in both train and test dataset
for col in x.columns:
    if x[col].dtype == 'object':
        imputer = SimpleImputer(strategy= 'most_frequent')
        imputer.fit(x[[col]])
        x[col] = imputer.transform(x[[col]]).ravel()
        test[col] = imputer.transform(test[[col]]).ravel()
    else:
        imputer = SimpleImputer(strategy= 'mean')
        imputer.fit(x[[col]])
        x[col] = imputer.transform(x[[col]]).ravel()
        test[col] = imputer.transform(test[[col]]).ravel()
        



In [24]:
onehot = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  # Set sparse=False to get a dense array

# For the training dataset
categorical_columns = [col for col in x.columns if x[col].dtype == 'object']  # Identify categorical columns
x_encoded = onehot.fit_transform(x[categorical_columns])  # Fit and transform only the categorical columns

# Replace the categorical columns with the one-hot encoded version
x = pd.concat([
    x.drop(columns=categorical_columns).reset_index(drop=True),  # Drop original categorical columns
    pd.DataFrame(x_encoded, columns=onehot.get_feature_names_out(categorical_columns))  # Add encoded columns
], axis=1)

# For the test dataset
test_encoded = onehot.transform(test[categorical_columns])  # Transform the test dataset with the same encoder

# Replace the categorical columns in the test dataset
test = pd.concat([
    test.drop(columns=categorical_columns).reset_index(drop=True),  # Drop original categorical columns
    pd.DataFrame(test_encoded, columns=onehot.get_feature_names_out(categorical_columns))  # Add encoded columns
], axis=1)

# Tuning hyperparameters

In [ ]:
# Slpit the dataset in to training dataset and validation dataset
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size= 0.2)

In [ ]:
# Split train dataset into smaller subset
x_train_subset, _, y_train_subset, _ = train_test_split(x_train, y_train, train_size= 0.2, random_state= 23)

In [27]:
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import f1_score


In [ ]:
# Hyperparameters of logistic regresion
params = {
    'penalty': ['l2'],
    'C': [0.001, 0.01, 1, 10],
    'solver': ['lbfgs', 'saga', 'newton-cg'],
    'class_weight': ['balanced']
}
max_iters = 100

In [29]:
params['penalty']

['l2']

In [30]:
# # Tuning hyperparameters
# best_f1_score = 0

# for c in params['c']:
#     for penalty in params['penalty']:
#         for solver in params['solver']:
#             for class_weight in params['class_weight']:
#                 if penalty == 'l1' and solver not in ['liblinear', 'saga']:
#                     continue
#                 if penalty == 'l2' and solver not in ['lbfgs', 'liblinear', 'saga', 'newton-cg']:
#                     continue
#                 try:
#                     model = LogisticRegression(C = c, penalty = penalty, solver = solver, class_weight = class_weight, max_iter= max_iters)
#                     model.fit(x_train, y_train)
#                     y_pred = model.predict(x_val)
#                     f1 = f1_score(y_val, y_pred)
#                     if f1 > best_f1_score:
#                         best_f1_score = f1
#                         best_params = {'C': c, 'penalty': penalty, 'solver': solver, 'class_weight': class_weight}
#                 except Exception as e:
#                     print(f"Skipping combination C={c}, penalty={penalty}, solver={solver}, class weight = {class_weight} due to error: {e}")

    

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [32]:

# from sklearn.model_selection import RandomizedSearchCV

# model = LogisticRegression(max_iter=500, tol = 1e-4)
# rand_search = RandomizedSearchCV(model, params, n_iter=5, scoring='f1')
# rand_search.fit(x_train_subset, y_train_subset)

# print("Best Parameters:", rand_search.best_params_)


In [33]:
# print("Best Parameters:", rand_search.best_params_)


In [44]:
# Scale the dataset
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scale = scaler.transform(x_train)
test_scale = scaler.transform(test)

In [ ]:
# Build model
true_model = LogisticRegression(max_iter= 2000, solver= 'saga', penalty= 'l1', class_weight= 'balanced', C = 0.01)
true_model.fit(x_train_scale, y_train)
result = true_model.predict_proba(test_scale)

In [41]:
true_model.classes_

array([0, 1], dtype=int64)

In [46]:
result

array([[0.23512474, 0.76487526],
       [0.78371251, 0.21628749],
       [0.37463404, 0.62536596],
       ...,
       [0.45867693, 0.54132307],
       [0.76567505, 0.23432495],
       [0.35382603, 0.64617397]])

In [47]:
submission_ver2 = pd.DataFrame({"SK_ID_CURR": test['SK_ID_CURR'].astype('int'), "TARGET": result[:, 1]})

In [48]:
len(submission_ver2)

61502

In [49]:
submission_ver2.sort_values('SK_ID_CURR', ascending= True, inplace = True)

In [ ]:
# Output result
submission_ver2.to_csv("D:/submission_ver2.csv", index=False , header = 1)